In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
from model import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale = 1./255)

train_image_generator = train_datagen.flow_from_directory(
'C:/Users/fullt/OneDrive/Documents/Research_Dir/Data/train_frames',
color_mode = 'grayscale',
class_mode = None,
batch_size = 4)

train_mask_generator = train_datagen.flow_from_directory(
'C:/Users/fullt/OneDrive/Documents/Research_Dir/Data/train_masks',
color_mode = 'grayscale',
class_mode = None,
batch_size = 4)

val_image_generator = val_datagen.flow_from_directory(
'C:/Users/fullt/OneDrive/Documents/Research_Dir/Data/val_frames',
color_mode = 'grayscale',
class_mode = None,
batch_size = 4)

val_mask_generator = val_datagen.flow_from_directory(
'C:/Users/fullt/OneDrive/Documents/Research_Dir/Data/val_masks',
color_mode = 'grayscale',
class_mode = None,
batch_size = 4)

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

NO_OF_TRAINING_IMAGES = len(os.listdir('C:/Users/fullt/OneDrive/Documents/Research_Dir/Data/train_frames/train/'))
NO_OF_VAL_IMAGES = len(os.listdir('C:/Users/fullt/OneDrive/Documents/Research_Dir/Data/val_frames/val/'))

NO_OF_EPOCHS = 100

BATCH_SIZE = 1

weights_path = 'C:/Users/fullt/OneDrive/Documents/Research_Dir/Unet_save/'

m = unet()
m.compile(loss = 'mse',
          optimizer = 'adam',
          metrics = [tf.keras.metrics.MeanIoU(num_classes=2)])

checkpoint = ModelCheckpoint(weights_path, monitor = 'mse', verbose = 1, save_best_only = True, mode = 'max')

csv_logger = CSVLogger('./log.out', append = True, separator = ';')

earlystopping = EarlyStopping(monitor = 'mse', verbose =1,
                             min_delta = 0.01, patience = 3, mode = 'max')

callbacks_list = [checkpoint, csv_logger, earlystopping]

results = m.fit(train_generator, epochs = NO_OF_EPOCHS,
                         steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                        validation_data=val_generator, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                         callbacks = callbacks_list)


m.save('Model.h5')

Found 254 images belonging to 1 classes.
Found 254 images belonging to 1 classes.
Found 73 images belonging to 1 classes.
Found 73 images belonging to 1 classes.
Epoch 1/100
  1/254 [..............................] - ETA: 33:07 - loss: 0.2490 - mean_io_u: 0.4565

KeyboardInterrupt: 

In [28]:
print(train_image_generator.next()[0].shape)
print(train_image_generator.next()[1].shape)

(4, 256, 256, 1)
(4, 1)
